In [ ]:
import os

import openai
from llama_index import (
    GPTSimpleVectorIndex,
    SimpleDirectoryReader,
)

from langchain.llms import OpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

In [ ]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
assert len(OPENAI_API_KEY) > 0

## ChatGPT

In [ ]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "ハバタクカミの対策を教えて"}]
)

In [ ]:
print("ChatGPTの回答:")
for choice in completion.choices:
    print("。\n".join(choice.message.content.split("。")))

## LlamaIndex

In [ ]:
documents = SimpleDirectoryReader('../data').load_data()
index = GPTSimpleVectorIndex(documents)

In [ ]:
index.save_to_disk('../tmp/index.json')

In [ ]:
response = index.query("ハバタクカミの対策を教えて")

In [ ]:
print("LlamaIndexの回答:")
print("。\n".join(response.response.split("。")))

## LangChain

In [ ]:
llm = OpenAI()

In [ ]:
loader = DirectoryLoader('../data/')

In [ ]:
docs = loader.load()

In [ ]:
len(docs)

In [ ]:
question_prompt_template = """長い文書の次の部分を使って、質問に答えるために関連するテキストがあるかどうかを確認してください。

{context}

質問: {question}
関連するテキスト:"""
QUESTION_PROMPT = PromptTemplate(
    template=question_prompt_template, input_variables=["context", "question"]
)

combine_prompt_template = """次のような長文の一部分を抜き出したものと質問を与えるので、最終的な答えを日本語で作成してください。
答えがわからなければ、わからないと言ってください。答えを作ろうとしないでください。

質問: {question}
=========
{summaries}
=========
答え:"""
COMBINE_PROMPT = PromptTemplate(
    template=combine_prompt_template, input_variables=["summaries", "question"]
)

In [ ]:
# chain = load_qa_chain(llm)
chain = load_qa_chain(llm, chain_type="map_reduce", return_map_steps=True, question_prompt=QUESTION_PROMPT, combine_prompt=COMBINE_PROMPT)

response = chain({"input_documents": docs, "question": "ハバタクカミの対策を教えて"})

In [ ]:
response

In [ ]:
print("LangChainの回答:")
print("。\n".join(response["output_text"].split("。")))